This is one of the deliverables produced from this project: https://library.hkust.edu.hk/ds/project/p002/
> Created by LAU Ming Kit, Jack (Year 4 student, BEng in Computer Engineering, HKUST)

In [30]:
# Import libraries
import torch, requests, csv
import pandas as pd
from PIL import ImageFile

# RAM
from PIL import Image
from ram.models import ram_plus
from ram import inference_ram as inference
from ram import get_transform

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# download the model from the URL below and put in the "pretrained" folder of your working directory 
# https://huggingface.co/xinyu1205/recognize-anything-plus-model/resolve/main/ram_plus_swin_large_14m.pth
ram_pth =  "pretrained/ram_plus_swin_large_14m.pth"
image_size = 384

#######load model
model = ram_plus(pretrained=ram_pth,
                            image_size=image_size,
                            vit='swin_l')
model.eval()

model = model.to(device)

ImageFile.LOAD_TRUNCATED_IMAGES = True

df = pd.read_csv('data.csv')
df

--------------
pretrained/ram_plus_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_plus_swin_large_14m.pth
vit: swin_l


,_name,aID,id,pid,_description,year,_URL,keyword
0,com_kite_001sm,32,evimg2/a1-10/COM_KITE_001SM.jpg,0,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"megaphone,catch,trumpet,crowd,person,instrumen..."
1,com_kite_002sm,32,evimg2/a1-10/COM_KITE_002SM.jpg,1,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"chair,crowd,event,foldingchair,collect,gatheri..."
2,com_kite_003sm,32,evimg2/a1-10/COM_KITE_003SM.jpg,2,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"catch,couple,field,gathering,grass,grassy,gree..."
3,com_kite_004sm,32,evimg2/a1-10/COM_KITE_004SM.jpg,3,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"fence,grass,person,sign,stand,woman"
4,com_kite_005sm,32,evimg2/a1-10/COM_KITE_005SM.jpg,4,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"building,child,cloudy,field,fly,girl,grass,gra..."
...,...,...,...,...,...,...,...,...
31797,2-2_057_02-2,982,evimg2/fm-vpaao-9912/2-2_057_02.jpg,32987,Visit of Dan Tsui to UST,2015,http://digitalimages.ust.hk/gallery/?p=1892,a
31798,2-2_057_03-2,982,evimg2/fm-vpaao-9912/2-2_057_03.jpg,32988,Visit of Dan Tsui to UST,2015,http://digitalimages.ust.hk/gallery/?p=1892,a
31799,2-2_057_04-2,982,evimg2/fm-vpaao-9912/2-2_057_04.jpg,32989,Visit of Dan Tsui to UST,2015,http://digitalimages.ust.hk/gallery/?p=1892,a
31800,2-2_057_05-2,982,evimg2/fm-vpaao-9912/2-2_057_05.jpg,32990,Visit of Dan Tsui to UST,2015,http://digitalimages.ust.hk/gallery/?p=1892,a


In [31]:
for i in range(6783, len(df)):
    try:
        # Global variable
        # Change this image path if needed
        image_pth = df.loc[i, 'id']
        response = requests.get("https://digitalimages.hkust.edu.hk/gallery/"+image_pth, stream=True)   

        # Prediction
        ori_image = Image.open(response.raw)

        transform = get_transform(image_size=image_size)
        with torch.no_grad():
            image = transform(ori_image).unsqueeze(0).to(device)

            res = inference(image, model)
        
        result = res[0].replace(' ', '').split('|')
        
        df.loc[i, 'keyword'] = '%s' % ','.join(map(str, result))
        
        print('processing:', i ,'/', len(df))
        break
    except Exception as e:
        print('error at', i)
        
        # df.to_csv('./db/data.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
        break
print('finished')
# df.to_csv('./db/data.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)


processing: 6783 / 31802
finished


In [23]:
df.head(5)

,_name,aID,id,pid,_description,year,_URL,keyword
0,com_kite_001sm,32,evimg2/a1-10/COM_KITE_001SM.jpg,0,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"megaphone,catch,trumpet,crowd,person,instrumen..."
1,com_kite_002sm,32,evimg2/a1-10/COM_KITE_002SM.jpg,1,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"chair,crowd,event,foldingchair,collect,gatheri..."
2,com_kite_003sm,32,evimg2/a1-10/COM_KITE_003SM.jpg,2,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"catch,couple,field,gathering,grass,grassy,gree..."
3,com_kite_004sm,32,evimg2/a1-10/COM_KITE_004SM.jpg,3,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"fence,grass,person,sign,stand,woman"
4,com_kite_005sm,32,evimg2/a1-10/COM_KITE_005SM.jpg,4,Kite Flying Contest,2012,http://digitalimages.ust.hk/gallery2/?p=639,"building,child,cloudy,field,fly,girl,grass,gra..."
